# Многопоточность в Python. Библиотеки threading и multiprocessing.

http://cs.mipt.ru/advanced_python/lessons/lab6.html

## Задание 1:

распараллелить чистку текста (колонки text). Функция чистки текста уже написана за вас. Вам нужно только установить все библиотеки (например, через анаконду)


https://anaconda.org/anaconda/nltk

In [15]:
# подрубаем нужные библиотеки
import os # для путей
import numpy as np # для массивов
import pandas as pd # для работы и хранения данных в ОП

import warnings 
warnings.filterwarnings('ignore') # все предупреждения офф - чтобы не мешали

from nltk.corpus import stopwords # частые слова
from nltk.stem import PorterStemmer # нлтк стеммер
from nltk.stem import WordNetLemmatizer # нлтк лемматизатор
from nltk import word_tokenize, sent_tokenize # для разбиения предложений на токены-слова
import re # библиотека для регулярных выражений

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bindarya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bindarya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

данные здесь: https://www.kaggle.com/c/dlinnlp-spring-2019-clf/data

In [23]:
train_df = pd.read_csv ('train.csv')
test_df = pd.read_csv ( 'test.csv')
df = pd.concat([train_df, test_df], axis=0, ignore_index=True).drop(['Unnamed: 0', 'label'], axis=1)
print(len(train_df), len(test_df), len(df))
df.head(10)

24871 5647 30518


,text,title
0,Economists generally agree: China must overhau...,China and Economic Reform: Xi Jinping’s Track ...
1,LONDON—British Prime Minister Theresa May said...,Trade to Be a Big Topic in Theresa May’s U.S. ...
2,Beaches come in all sorts of shapes and sizes ...,"The Top Beaches In The World, According To Nat..."
3,"A timeline of what happened after Tamir Rice, ...",Sheriff’s Report Provides New Details on Tamir...
4,An Italian neurosurgeon who has claimed for mo...,Surgeon claiming he will transplant volunteer'...
5,Spanx does not do the thing I thought it did. ...,This Is How Differently Priced Spanx Can Actua...
6,"“Jane Pauley! You’re all in leather,” said Sam...",Samantha Bee and Jane Pauley Are Breaking the ...
7,Charles Krauthammer called President Trump's 5...,Krauthammer: Syria Strike 'Total Contradiction...
8,The United States is making more things than e...,Rust Belt voters made Trump president. Now the...
9,As Jewish settlers and protesters were removed...,"As Illegal Outpost Cleared, Israeli PM Netanya..."


In [17]:
# функция для очистки текста

def clean_text(data):
    low = True 
    delstops = True
    lemm = True
    stem = True
    del12gram = True  # True = убираем обрывки слов в 1-2 символа
    
    
    stops = set(stopwords.words("english"))
    data = str(data)
    data = re.sub(r'[^a-zA-Z ]',r' ',data)
    data = ' '.join(data.split()) #убираем лишние пробелы
    
    if low:
        data = data.lower()
    
    if delstops:
        data = " ".join([w for w in data.split() if w not in stops])

    if lemm:      
        wordnet_lemmatizer = WordNetLemmatizer()
        data = " ".join([wordnet_lemmatizer.lemmatize(w, pos='a') for w in data.split()])
        data = " ".join([wordnet_lemmatizer.lemmatize(w, pos='n') for w in data.split()])
        
    if stem:
        st = PorterStemmer()
        data = " ".join([st.stem(w) for w in data.split()])
        
    if del12gram:
        data = " ".join([w for w in data.split() if len(w) > 2])  
          
    return data

In [21]:
%%time
# применяем функцию очистки текста
df['cleaned_text'] = df['text'].map(clean_text) # ~ Wall time: 5min 34s

CPU times: user 9min 13s, sys: 30.4 s, total: 9min 43s
Wall time: 10min 32s


In [22]:
df.head(10)

,text,title,cleaned_text
0,Economists generally agree: China must overhau...,China and Economic Reform: Xi Jinping’s Track ...,economist gener agre china must overhaul huge ...
1,LONDON—British Prime Minister Theresa May said...,Trade to Be a Big Topic in Theresa May’s U.S. ...,london british prime minist theresa may said d...
2,Beaches come in all sorts of shapes and sizes ...,"The Top Beaches In The World, According To Nat...",beach come sort shape size beyond typic caribb...
3,"A timeline of what happened after Tamir Rice, ...",Sheriff’s Report Provides New Details on Tamir...,timelin happen tamir rice year old boy kill po...
4,An Italian neurosurgeon who has claimed for mo...,Surgeon claiming he will transplant volunteer'...,italian neurosurgeon claim month perform world...
5,Spanx does not do the thing I thought it did. ...,This Is How Differently Priced Spanx Can Actua...,spanx thing thought kristin like mani ladi lil...
6,"“Jane Pauley! You’re all in leather,” said Sam...",Samantha Bee and Jane Pauley Are Breaking the ...,jane pauley leather said samantha bee pauley w...
7,Charles Krauthammer called President Trump's 5...,Krauthammer: Syria Strike 'Total Contradiction...,charl krauthamm call presid trump missil strik...
8,The United States is making more things than e...,Rust Belt voters made Trump president. Now the...,unit state make thing ever read right manufact...
9,As Jewish settlers and protesters were removed...,"As Illegal Outpost Cleared, Israeli PM Netanya...",jewish settler protest remov illeg outpost occ...


In [27]:
from multiprocessing import Process

if __name__ == '__main__':
    clean = []
    p = Process(target=clean_text, args=(df['text'],))
    clean.append(p)
    p.start()
    p.join()


In [28]:
print(clean)

[<Process(Process-5, stopped)>]


In [25]:
df.head(10)

,text,title
0,Economists generally agree: China must overhau...,China and Economic Reform: Xi Jinping’s Track ...
1,LONDON—British Prime Minister Theresa May said...,Trade to Be a Big Topic in Theresa May’s U.S. ...
2,Beaches come in all sorts of shapes and sizes ...,"The Top Beaches In The World, According To Nat..."
3,"A timeline of what happened after Tamir Rice, ...",Sheriff’s Report Provides New Details on Tamir...
4,An Italian neurosurgeon who has claimed for mo...,Surgeon claiming he will transplant volunteer'...
5,Spanx does not do the thing I thought it did. ...,This Is How Differently Priced Spanx Can Actua...
6,"“Jane Pauley! You’re all in leather,” said Sam...",Samantha Bee and Jane Pauley Are Breaking the ...
7,Charles Krauthammer called President Trump's 5...,Krauthammer: Syria Strike 'Total Contradiction...
8,The United States is making more things than e...,Rust Belt voters made Trump president. Now the...
9,As Jewish settlers and protesters were removed...,"As Illegal Outpost Cleared, Israeli PM Netanya..."


# Задание 2

Иногда бывает нужно узнать доступность набора ip адресов. Неэффективный вариант представлен ниже.

Реализуйте то же самое, но используя threading.

In [1]:
import os, re


In [2]:
%%time
received_packages = re.compile(r"(\d) received")
status = ("no response", "alive but losses", "alive")

def ip(n,m):
    for suffix in range(n, m):
        ip = "192.168.178." + str(suffix)
        ping_out = os.popen("ping -q -c2 " + ip, "r")  # получение вердикта
        print("... pinging ", ip)
        while True:
            line = ping_out.readline()
            if not line:
                break
            n_received = received_packages.findall(line)
            if n_received:
                print(ip + ": " + status[int(n_received[0])])
ip(20,30)

... pinging  192.168.178.20
192.168.178.20: no response
... pinging  192.168.178.21
192.168.178.21: no response
... pinging  192.168.178.22
192.168.178.22: no response
... pinging  192.168.178.23
192.168.178.23: no response
... pinging  192.168.178.24
192.168.178.24: no response
... pinging  192.168.178.25
192.168.178.25: no response
... pinging  192.168.178.26
192.168.178.26: no response
... pinging  192.168.178.27
192.168.178.27: no response
... pinging  192.168.178.28
192.168.178.28: no response
... pinging  192.168.178.29
192.168.178.29: no response
CPU times: user 141 ms, sys: 406 ms, total: 547 ms
Wall time: 1min 52s


In [3]:
import threading
import sys

In [4]:
def run_threads(count):
    threads = [
        threading.Thread(target=ip, args=(20,30))
        
    ]
    for thread in threads:
        thread.start()  # каждый поток должен быть запущен
    for thread in threads:
        thread.join()  # дожидаемся исполнения всех потоков

In [5]:
%%time
run_threads(4)
print('finish')

... pinging  192.168.178.20
192.168.178.20: no response
... pinging  192.168.178.21
192.168.178.21: no response
... pinging  192.168.178.22
192.168.178.22: no response
... pinging  192.168.178.23
192.168.178.23: no response
... pinging  192.168.178.24
192.168.178.24: no response
... pinging  192.168.178.25
192.168.178.25: no response
... pinging  192.168.178.26
192.168.178.26: no response
... pinging  192.168.178.27
192.168.178.27: no response
... pinging  192.168.178.28
192.168.178.28: no response
... pinging  192.168.178.29
192.168.178.29: no response
finish
CPU times: user 46.9 ms, sys: 266 ms, total: 312 ms
Wall time: 1min 52s


# Задание 3

= Упражнение №4 из лабы про мультипроцессинг

In [8]:
import time
import urllib.request

urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read(urls):
    for url in urls:
        with urllib.request.urlopen(url) as u:
            return u.read()


start = time.time()
read(urls)
print(time.time() - start)

0.9187958240509033


In [9]:
from multiprocessing import Process



start = time.time()
if __name__ == '__main__':
    p = Process(target= read, args=(urls,))
    p.start()
    p.join()

print(time.time() - start)

0.5492768287658691


дедлайн по ДЗ8 - вечер следующего вс 3.11.2019 23:59. Удачи!